## Object Relational Tutorial (1.x API)

The `SQLAlchemy Object Relational Mapper` presents a method of _associating_ __user-defined `Python classes` with `database tables`__, and _instances_ of those classes (_objects_) with `rows` in their corresponding `tables`. It _includes_ a system that __transparently synchronizes all changes in state__ between _objects_ and their _related rows_, called a `unit of work`, as well as a system for _expressing database queries_ in terms of the `user defined classes` and their `defined relationships` between each other.

The `ORM` is in _contrast_ to the `SQLAlchemy Expression Language`, upon which the `ORM` is constructed. Whereas the `SQL Expression Language`, introduced in `SQL Expression Language Tutorial (1.x API)`, presents a system of _representing_ the __primitive constructs of the relational database directly__ _without opinion_, the `ORM` presents a __high level__ and __abstracted__ `pattern of usage`, which itself is an example of applied usage of the _Expression Language_.

While there is __overlap__ among the `usage patterns` of the `ORM` and the `Expression Language`, the __similarities are more superficial__ than they may at first appear. One approaches the _structure and content of data_ from the `perspective` of a __`user-defined domain model`__ which is __transparently persisted and refreshed__ from its _underlying storage model_. The other approaches it from the perspective of __`literal schema` and `SQL expression` representations__ which are __explicitly composed into messages__ _consumed individually by the database_.

A _successful_ `application` may be constructed using the `Object Relational Mapper` __exclusively__. In _advanced situations_, an application constructed with the ORM may make _occasional usage_ of the `Expression Language` __directly__ in certain areas where __specific database interactions are required__.

#### Version Check

A _quick check_ to __verify__ that we are on at `least version 1.4` of `SQLAlchemy`.

In [1]:
import sqlalchemy
from sqlalchemy import Column, Integer, String, create_engine
from sqlalchemy.orm import sessionmaker, declarative_base, aliased

In [2]:
sqlalchemy.__version__

'1.4.41'

#### Connecting

For this notebook we will use an __in-memory-only__ `SQLite database`. To connect we use `create_engine()`.

In [3]:
engine = create_engine("sqlite:///:memory:", echo=True)

The `echo` flag is a _shortcut_ to __setting up `SQLAlchemy` logging__, which is accomplished via __Python's standard `logging` module__. With it enabled, we'll see _all the generated SQL produced_. If you are working through this notebook and want _less output generated_, set it to `False`. This notebook will _format_ the `SQL` __behind a popup window__ so it doesn't get in our way; just click the `"SQL"` links to see what's being _generated_.

The _return value_ of `create_engine()` is an __instance of `Engine`__, and it _represents_ the __core interface to the database__, adapted through a `dialect` that _handles the details of the database and DBAPI in use_. In this case the `SQLite dialect` will __interpret instructions__ to the Python _built-in_ `sqlite3` module.

> ##### Lazy Connecting
> The `Engine`, when first returned by `create_engine()`, has __not actually tried to connect__ to the database yet; that _happens only the first time it is asked to perform a task_ against the database.

The first time a method like `Engine.execute()` or `Engine.connect()` is called, the `Engine` _establishes_ a __real `DBAPI` connection__ to the database, which is then __used to emit the SQL__. When using the `ORM`, we typically __don't use the `Engine` directly once created__; instead, it's _used behind the scenes_ by the `ORM` as we'll see shortly.

#### Declare a Mapping

When using the `ORM`, the _configurational process_ __starts by `describing` the database tables__ we'll be dealing with, and then by __defining__ our own `classes` which will be _mapped_ to those `tables`. In _modern SQLAlchemy_, these `two tasks` are _usually_ __performed together__, using a system known as `Declarative Extensions`, which allows us to _create classes_ that __include directives to describe__ the `actual database table` they will be _mapped to_.

Classes _mapped_ using the `Declarative system` are _defined_ in terms of a __base class__ which _maintains_ a __catalog of classes and tables relative to that base__ - this is known as the __`declarative base class`__. Our application will _usually_ have __just one instance of this base__ in a _commonly imported module_. We _create_ the `base class` using the `declarative_base()` function, as follows.

In [4]:
Base = declarative_base()

Now that we have a __`"base"`__, we __can define__ _any number of_ `mapped classes` in terms of it. We will start with just a single table called `users`, which will _store records_ for the end-users using our application. A new class called `User` will be the class to which we _map_ this `table`. _Within the class_, we _define_ __details about the table__ to which we'll be __mapping__, _primarily_ the `table name`, and `names` and `datatypes of columns`.

In [5]:
class User(Base):
    __tablename__ = "users"
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    nickname = Column(String)
    
    def __repr__(self):
        return f"<User(name={self.name}, fullname={self.fullname}, nickname={self.nickname})>"

> ##### Tip
> The `User` class defines a `__repr__()` method, but note that is __optional__; we only _implement_ it in this notebook so that our examples show _nicely formatted_ `User` _objects_.

A class using `Declarative` at a _minimum_ __needs a `__tablename__` attribute__, and __at least one `Column`__ which is __part of a `primary key`__. `SQLAlchemy` __never makes any assumptions__ by itself _about the table to which a class refers_, including that it has __no built-in conventions__ for _names, datatypes, or constraints_. But this __doesn't mean boilerplate is required__; instead, you're encouraged to _create_ your __own automated conventions__ using `helper functions` and `mixin classes`, which is described in detail at `Mixin and Custom Base Classes`.

When our class is constructed, `Declarative` __replaces all the `Column` objects__ with _special_ `Python accessors` known as __`descriptors`__; this is a process known as __`instrumentation`__. The `"instrumented"` _mapped class_ will provide us with the means to _refer to our table_ in a `SQL context` as well as to __persist and load the values of columns__ from the database.

__Outside of what the `mapping process` does to our class, the class remains otherwise mostly a `normal Python class`__, to which we can _define any number of ordinary attributes and methods_ needed by our application.

#### Create a Schema

With our `User` class _constructed via_ the __Declarative system__, we have _defined information_ about our `table`, known as __`table metadata`__. The object used by `SQLAlchemy` to _represent this information for a specific table_ is called the `Table object`, and here `Declarative` has made one for us. We can see this object by inspecting the `__table__` _attribute_.

In [6]:
User.__table__

Table('users', MetaData(), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('name', String(), table=<users>), Column('fullname', String(), table=<users>), Column('nickname', String(), table=<users>), schema=None)

> ##### Classical Mappings
> The `Declarative system`, though _highly recommended_, is __not required__ in order to use _SQLAlchemy's ORM_. _Outside of Declarative_, any _plain Python class_ __can be mapped to any `Table`__ using the `mapper()` function __directly__; this _less common usage_ is described at `Imperative Mapping`.

When we declared our class, `Declarative` used a __Python metaclass__ in order to _perform additional activities_ once the class declaration was __complete__; within this phase, it then _created_ a `Table object` __according to our specifications__, and __associated it with the class__ by constructing a `Mapper object`. This object is a __behind-the-scenes__ object we _normally don't need to deal with directly_ (though it can _provide plenty of information_ about our _mapping_ when we need it).

The `Table` object is a _member_ of a _larger collection_ known as __`MetaData`__. When using `Declarative`, this object is _available_ using the `.metadata` attribute of our _declarative base class_.

The `MetaData` is a __registry__ which includes the ability to _emit a limited set of schema generation commands_ to the database. As our `SQLite` database __does not actually have a users table present__, we can use `MetaData` to _issue_ `CREATE TABLE` statements to the database _for all tables that don't yet exist_. Below, we call the `MetaData.create_all()` method, passing in our `Engine` as a __source of database connectivity__. We will see that _special commands are first emitted_ to _check_ for the __presence of the users table__, and following that the __actual `CREATE TABLE` statement__.

In [7]:
Base.metadata.create_all(engine)

2022-10-16 11:42:03,439 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-16 11:42:03,441 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2022-10-16 11:42:03,442 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-10-16 11:42:03,445 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("users")
2022-10-16 11:42:03,447 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-10-16 11:42:03,450 INFO sqlalchemy.engine.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	nickname VARCHAR, 
	PRIMARY KEY (id)
)


2022-10-16 11:42:03,452 INFO sqlalchemy.engine.Engine [no key 0.00180s] ()
2022-10-16 11:42:03,454 INFO sqlalchemy.engine.Engine COMMIT


> ##### Minimal Table Descriptions vs. Full Descriptions
> 
> Users familiar with the _syntax_ of `CREATE TABLE` may notice that the _VARCHAR columns were generated_ __without a length__; on `SQLite` and `PostgreSQL`, this is a _valid datatype_, but on _others_, it's _not allowed_. So if running this notebook on one of those databases, and you wish to use `SQLAlchemy` to issue `CREATE TABLE`, a `"length"` may be provided to the `String` type as below:
> 
> `Column(String(50))`
> 
> The _length_ field on `String`, as well as _similar_ `precision/scale` fields available on `Integer`, `Numeric`, etc. are __not referenced__ by `SQLAlchemy` other than when _creating tables_.
> 
> Additionally, `Firebird` and `Oracle` __require `sequences` to generate new primary key identifiers__, and `SQLAlchemy` __doesn't generate or assume__ these _without_ being __instructed__. For that, you use the `Sequence` construct.
> 
> ```
> from sqlalchemy import Sequence
> Column(Integer, Sequence("user_id_seq"), primary_key=True)
> ```
> 
> A _full, foolproof_ `Table` generated via our `declarative mapping` is therefore:
> 
> ```
> class User(Base):
>     __tablename__ = "users"
>     id = Column(Integer, Sequence("user_id_seq"), primary_key=True)
>     name = Column(String(50))
>     fullname = Column(String(50))
>     nickname = Column(String(50))
> 
>     def __repr__(self):
>         return "<User(name='%s', fullname='%s', nickname='%s')>" % (
>             self.name,
>             self.fullname,
>             self.nickname,
>         )
> ```
> 
> We include this _more verbose table definition_ __separately__ to __`highlight`__ the _difference between a minimal construct_ __geared__ _primarily_ towards __in-Python usage only__, versus one that will be _used to emit_ `CREATE TABLE` statements on a _particular set of backends_ with __more stringent requirements__.

#### Create an Instance of the Mapped Class

_With mappings complete_, let's now __create and inspect__ a `User` object.

In [8]:
ed_user = User(name="ed", fullname="Ed Jones", nickname="edsnickname")
print(f"{ed_user.name = }, {ed_user.nickname = }, {ed_user.id = }")

ed_user.name = 'ed', ed_user.nickname = 'edsnickname', ed_user.id = None


> ##### the `__init__()` method
> Our `User` class, as defined using the `Declarative system`, has been _provided with a constructor_ (e.g. `__init__()` method) which __automatically accepts keyword names that match the columns we've mapped__. We are _free to define_ any __explicit `__init__()` method__ we prefer on our class, which will __override the default method__ provided by `Declarative`.

Even though we __didn't specify__ it in the _constructor_, the `id` attribute __still produces a value of `None`__ when we _access_ it (_as opposed to Python's usual behavior_ of raising `AttributeError` for an _undefined attribute_). `SQLAlchemy`'s __instrumentation__ normally _produces_ this _default value_ for `column-mapped attributes` __when first accessed__. For those attributes where we've _actually assigned a value_, the _instrumentation system_ is __tracking those assignments__ for use _within_ an eventual `INSERT statement` to be __emitted__ to the database.

#### Creating a Session

We're now ready to start talking to the database. The _ORM_'s `"handle"` to the database is the `Session`. When we _first_ __set up the application__, at the _same level_ as our `create_engine()` statement, we define a `Session` class which will __serve as a factory__ for _new_ `Session` objects.

In [9]:
Session = sessionmaker(bind=engine)

In the case where your _application_ __does not yet have an `Engine`__ when you `define` your _module-level_ objects, just set it up like this.

In [10]:
Session = sessionmaker()

Later, when you _create your engine_ with `create_engine()`, __connect it to the `Session`__ using `sessionmaker.configure()`.

In [11]:
Session.configure(bind=engine)

> ##### Session life-cycle patterns
> The question of when to make a `Session` _depends_ a lot on __what kind of application__ is being built. Keep in mind, the `Session` is just a __workspace for your objects__, _local to a particular database connection_ - if you think of an _application thread_ as a guest at a dinner party, the `Session` is the guest's plate and the objects it holds are the food (and the database...the kitchen?)! More on this topic available at `When do I construct a Session, when do I commit it, and when do I close it?`.

This _custom-made_ `Session` class will __create new Session objects__ which are __bound to our database__. Other __`transactional characteristics`__ may be _defined_ when calling `sessionmaker` as well; these are described in a later chapter. Then, whenever you need to _have a conversation with the database_, you __instantiate__ a `Session`.

In [12]:
session = Session()

The above `Session` is __associated__ with our _SQLite-enabled_ `Engine`, but it __hasn't opened any connections__ yet. When it's _first used_, it __retrieves a connection__ from a __pool of connections__ _maintained by_ the `Engine`, and _holds onto it_ until we __`commit` all changes__ _and/or_ __close__ the `session` object.

#### Adding and Updating Objects

To __persist__ our `User` object, we `Session.add()` it to our `Session`.

In [13]:
ed_user = User(name="ed", fullname="Ed Jones", nickname="edsnickname")
session.add(ed_user)

_At this point_, we say that the _instance_ is `pending`; __no SQL has yet been issued__ and the _object_ is __not yet represented by a row in the database__. The `Session` will __issue the SQL to persist__ Ed Jones _as soon as is needed_, using a _process_ known as a `flush`. If we _query the database for Ed Jones_, __all pending information will first be flushed__, and the _query is issued_ __immediately thereafter__.

For example, below we _create a new_ `Query` _object_ which __loads instances of `User`__. We __`"filter by"`__ the _name attribute of ed_, and _indicate_ that we'd like __only the first result__ in the _full list of rows_. A `User` instance is returned which is __equivalent to that which we've added__.

In [14]:
our_user = session.query(User).filter_by(name="ed").first()
print(f"{our_user = }")

2022-10-16 11:42:04,324 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-16 11:42:04,328 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2022-10-16 11:42:04,330 INFO sqlalchemy.engine.Engine [generated in 0.00200s] ('ed', 'Ed Jones', 'edsnickname')
2022-10-16 11:42:04,335 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?
2022-10-16 11:42:04,337 INFO sqlalchemy.engine.Engine [generated in 0.00150s] ('ed', 1, 0)
our_user = <User(name=ed, fullname=Ed Jones, nickname=edsnickname)>


In fact, the `Session` has __identified__ that the _row returned is the same row_ as one __already represented__ within its `internal map of objects`, so we __actually got back__ the __`identical instance`__ as that which we _just added_.

In [15]:
print(f"{ed_user is our_user = }")

ed_user is our_user = True


The _ORM concept at work_ here is known as an __`identity map`__ and _ensures_ that __`all operations` upon a particular row within a `Session` operate upon the same set of data__. Once an object with a particular _primary key_ is __present__ in the `Session`, _all SQL queries_ on that `Session` will always __return the `same` Python object__ for that _particular primary key_; it also will __raise an error__ if an _attempt is made_ to place a _second, already-persisted object_ with the __same primary key__ _within_ the `session`.

We can _add more_ `User` objects __at once__ using `add_all()`.

In [16]:
session.add_all(
    [
        User(name="wendy", fullname="Wendy Williams", nickname="windy"),
        User(name="mary", fullname="Mary Contrary", nickname="mary"),
        User(name="fred", fullname="Fred Flintstone", nickname="freddy"),
    ]
)

Also, we've decided `Ed`'s _nickname_ isn't that great, so let's __change__ it.

In [17]:
ed_user.nickname = "eddie"

The `Session` is __paying attention__. It _knows_, for example, that `Ed Jones` has been __modified__.

In [18]:
session.dirty

IdentitySet([<User(name=ed, fullname=Ed Jones, nickname=eddie)>])

and that _three new_ `User` objects are __pending__.

In [19]:
session.new

IdentitySet([<User(name=wendy, fullname=Wendy Williams, nickname=windy)>, <User(name=mary, fullname=Mary Contrary, nickname=mary)>, <User(name=fred, fullname=Fred Flintstone, nickname=freddy)>])

We tell the `Session` that we'd like to _issue all remaining changes_ to the database and __commit the transaction__, which has been __in progress__ _throughout_. We do this via `Session.commit()`. The `Session` __emits the `UPDATE` statement__ for the _nickname change_ on `"ed"`, as well as `INSERT` statements for the _three new_ `User` objects we've __added__.

In [20]:
session.commit()

2022-10-16 11:42:05,050 INFO sqlalchemy.engine.Engine UPDATE users SET nickname=? WHERE users.id = ?
2022-10-16 11:42:05,051 INFO sqlalchemy.engine.Engine [generated in 0.00148s] ('eddie', 1)
2022-10-16 11:42:05,053 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2022-10-16 11:42:05,054 INFO sqlalchemy.engine.Engine [cached since 0.7258s ago] ('wendy', 'Wendy Williams', 'windy')
2022-10-16 11:42:05,055 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2022-10-16 11:42:05,056 INFO sqlalchemy.engine.Engine [cached since 0.7278s ago] ('mary', 'Mary Contrary', 'mary')
2022-10-16 11:42:05,057 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2022-10-16 11:42:05,058 INFO sqlalchemy.engine.Engine [cached since 0.7299s ago] ('fred', 'Fred Flintstone', 'freddy')
2022-10-16 11:42:05,059 INFO sqlalchemy.engine.Engine COMMIT


`Session.commit()` __flushes__ the _remaining changes_ to the database, and __commits the transaction__. The `connection resources` _referenced by_ the `session` are now __returned to the `connection pool`__. _Subsequent operations_ with this `session` will occur in a __new transaction__, which will again __re-acquire__ _connection resources when first needed_.

If we look at `Ed`'s _id attribute_, which _earlier_ was `None`, it __now has a value__.

In [21]:
ed_user.id

2022-10-16 11:42:05,213 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-16 11:42:05,215 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.id = ?
2022-10-16 11:42:05,217 INFO sqlalchemy.engine.Engine [generated in 0.00161s] (1,)


1

After the `Session` _inserts new rows_ in the database, __all newly generated `identifiers` and `database-generated defaults` become available on the instance__, either _immediately or via load-on-first-access_. In this case, the __entire row was `re-loaded` on access__ because a _new transaction_ was begun _after_ we issued `Session.commit()`. `SQLAlchemy` _by default_ __refreshes data from a previous transaction__ the _first time it's accessed_ within a `new transaction`, so that the __most recent state__ is _available_. The _level of reloading_ is __`configurable`__ as is described in `Using the Session`.

> ##### Session Object States
> As our `User` object _moved_ from being __outside the `Session`__, to __inside the `Session`__ _without a primary key_, __to actually being inserted__, it __moved between three out of five available "object states"__ - `transient`, `pending`, and `persistent`. _Being aware of these_ __`states`__ and _what they mean_ is always a good idea - be sure to read `Quickie Intro to Object States` for a quick overview.

#### Rolling Back

Since the `Session` __works within a transaction__, we can __roll back changes made__ too. Let's make _two changes_ that we'll __revert__; `ed_user`'s user _name_ gets set to `Edwardo`.

In [22]:
ed_user.name = "Edwardo"

and we'll _add_ another _erroneous user_, `fake_user`.

In [23]:
fake_user = User(name="fakeuser", fullname="Invalid", nickname="12345")
session.add(fake_user)

_Querying_ the `session`, we can see that they're __flushed__ into the _current transaction_.

In [24]:
session.query(User).filter(User.name.in_(["Edwardo", "fakeuser"])).all()

2022-10-16 11:42:05,559 INFO sqlalchemy.engine.Engine UPDATE users SET name=? WHERE users.id = ?
2022-10-16 11:42:05,562 INFO sqlalchemy.engine.Engine [generated in 0.00168s] ('Edwardo', 1)
2022-10-16 11:42:05,563 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2022-10-16 11:42:05,564 INFO sqlalchemy.engine.Engine [cached since 1.236s ago] ('fakeuser', 'Invalid', '12345')
2022-10-16 11:42:05,567 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name IN (?, ?)
2022-10-16 11:42:05,569 INFO sqlalchemy.engine.Engine [generated in 0.00257s] ('Edwardo', 'fakeuser')


[<User(name=Edwardo, fullname=Ed Jones, nickname=eddie)>,
 <User(name=fakeuser, fullname=Invalid, nickname=12345)>]

__Rolling back__, we can see that `ed_user`'s _name_ is back to `ed`, and `fake_user` has been _kicked out_ of the `session`.

In [25]:
session.rollback()
print(f"{ed_user.name = }")
print(f"{fake_user in session = }")

2022-10-16 11:42:05,718 INFO sqlalchemy.engine.Engine ROLLBACK
2022-10-16 11:42:05,720 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-16 11:42:05,721 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.id = ?
2022-10-16 11:42:05,722 INFO sqlalchemy.engine.Engine [cached since 0.5074s ago] (1,)
ed_user.name = 'ed'
fake_user in session = False


_issuing_ a `SELECT` __illustrates the changes made__ to the database.

In [26]:
session.query(User).filter(User.name.in_(["ed", "fakeuser"])).all()

2022-10-16 11:42:05,834 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name IN (?, ?)
2022-10-16 11:42:05,835 INFO sqlalchemy.engine.Engine [cached since 0.2693s ago] ('ed', 'fakeuser')


[<User(name=ed, fullname=Ed Jones, nickname=eddie)>]

#### Querying

A `Query` object is _created_ using the `query()` method on `Session`. This function takes a _variable number of arguments_, which can be __any combination of classes__ _and_ __class-instrumented descriptors__. Below, we _indicate_ a `Query` which _loads_ `User` instances. When _evaluated_ in an __iterative context__, the _list of_ `User` _objects_ present is returned.

In [27]:
for instance in session.query(User).order_by(User.id):
    print(instance.name, instance.fullname)

2022-10-16 11:42:05,964 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users ORDER BY users.id
2022-10-16 11:42:05,966 INFO sqlalchemy.engine.Engine [generated in 0.00149s] ()
ed Ed Jones
wendy Wendy Williams
mary Mary Contrary
fred Fred Flintstone


The `Query` also _accepts_ __ORM-instrumented descriptors__ as arguments. _Any time multiple class entities_ or _column-based entities_ are expressed as arguments to the `query()` function, the __return result is expressed as tuples__.

In [28]:
for name, fullname in session.query(User.name, User.fullname):
    print(name, fullname)

2022-10-16 11:42:06,087 INFO sqlalchemy.engine.Engine SELECT users.name AS users_name, users.fullname AS users_fullname 
FROM users
2022-10-16 11:42:06,089 INFO sqlalchemy.engine.Engine [generated in 0.00163s] ()
ed Ed Jones
wendy Wendy Williams
mary Mary Contrary
fred Fred Flintstone


The _tuples_ returned by `Query` are __`named tuples`__, supplied by the `Row` class, and can be __treated much like an ordinary Python object__. The _names_ are the __same__ as the _attribute's name_ for an _attribute_, and the _class name_ for a `class`.

In [29]:
for row in session.query(User, User.name).all():
    print(row.User, row.name)

2022-10-16 11:42:06,216 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname, users.name AS users_name__1 
FROM users
2022-10-16 11:42:06,218 INFO sqlalchemy.engine.Engine [generated in 0.00149s] ()
<User(name=ed, fullname=Ed Jones, nickname=eddie)> ed
<User(name=wendy, fullname=Wendy Williams, nickname=windy)> wendy
<User(name=mary, fullname=Mary Contrary, nickname=mary)> mary
<User(name=fred, fullname=Fred Flintstone, nickname=freddy)> fred


You __can control__ the _names of individual column expressions_ using the `ColumnElement.label()` construct, which is __available__ _from any_ `ColumnElement-derived object`, as well as any `class attribute` which is _mapped to one_ (such as `User.name`).

In [30]:
for row in session.query(User.name.label("name_label")).all():
    print(row.name_label)

2022-10-16 11:42:06,335 INFO sqlalchemy.engine.Engine SELECT users.name AS name_label 
FROM users
2022-10-16 11:42:06,336 INFO sqlalchemy.engine.Engine [generated in 0.00109s] ()
ed
wendy
mary
fred


The name given to a _full entity_ such as `User`, _assuming_ that __multiple entities are present__ in the call to `Session.query()`, can be controlled using `aliased()`.

In [31]:
user_alias = aliased(User, name="user_alias")

for row in session.query(user_alias, user_alias.name).all():
    print(row.user_alias)

2022-10-16 11:42:06,460 INFO sqlalchemy.engine.Engine SELECT user_alias.id AS user_alias_id, user_alias.name AS user_alias_name, user_alias.fullname AS user_alias_fullname, user_alias.nickname AS user_alias_nickname, user_alias.name AS user_alias_name__1 
FROM users AS user_alias
2022-10-16 11:42:06,462 INFO sqlalchemy.engine.Engine [generated in 0.00115s] ()
<User(name=ed, fullname=Ed Jones, nickname=eddie)>
<User(name=wendy, fullname=Wendy Williams, nickname=windy)>
<User(name=mary, fullname=Mary Contrary, nickname=mary)>
<User(name=fred, fullname=Fred Flintstone, nickname=freddy)>


_Basic operations_ with `Query` include __issuing LIMIT and OFFSET__, _most conveniently_ __using Python `array slices`__ and _typically_ __in conjunction with `ORDER BY`__.

In [32]:
for u in session.query(User).order_by(User.id)[1:3]:
    print(u)

2022-10-16 11:42:06,608 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users ORDER BY users.id
 LIMIT ? OFFSET ?
2022-10-16 11:42:06,610 INFO sqlalchemy.engine.Engine [generated in 0.00146s] (2, 1)
<User(name=wendy, fullname=Wendy Williams, nickname=windy)>
<User(name=mary, fullname=Mary Contrary, nickname=mary)>


and __filtering results__, which is _accomplished_ either with `filter_by()`, which _uses keyword arguments_.

In [33]:
for (name,) in session.query(User.name).filter_by(fullname="Ed Jones"):
    print(name)

2022-10-16 11:42:06,742 INFO sqlalchemy.engine.Engine SELECT users.name AS users_name 
FROM users 
WHERE users.fullname = ?
2022-10-16 11:42:06,744 INFO sqlalchemy.engine.Engine [generated in 0.00212s] ('Ed Jones',)
ed


...or `filter()`, which uses __more flexible__ `SQL expression language` constructs. These allow you to _use regular Python operators_ with the _class-level attributes_ on your `mapped class`.

In [34]:
for (name,) in session.query(User.name).filter(User.fullname == "Ed Jones"):
    print(name)

2022-10-16 11:42:06,886 INFO sqlalchemy.engine.Engine SELECT users.name AS users_name 
FROM users 
WHERE users.fullname = ?
2022-10-16 11:42:06,887 INFO sqlalchemy.engine.Engine [cached since 0.1451s ago] ('Ed Jones',)
ed


The `Query` object is __fully generative__, meaning that _most method calls_ __return a new `Query` object upon which `further criteria` may be added__. For example, to _query_ for users _named_ `"ed"` with a _full name_ of `"Ed Jones"`, you can call `filter()` __twice__, which __joins criteria using `AND`__.

In [35]:
for user in (
    session.query(User).
    filter(User.name == "ed").
    filter(User.fullname == "Ed Jones")
):
    print(user)

2022-10-16 11:42:07,032 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ? AND users.fullname = ?
2022-10-16 11:42:07,034 INFO sqlalchemy.engine.Engine [generated in 0.00136s] ('ed', 'Ed Jones')
<User(name=ed, fullname=Ed Jones, nickname=eddie)>


##### Common Filter Operators

Here's a _rundown_ of some of the _most common operators_ used in `filter()`.

* __`ColumnOperators.__eq__()`__:

```
query.filter(User.name == "ed")
```

* __`ColumnOperators.__ne__()`__:

```
query.filter(User.name != "ed")
```

* __`ColumnOperators.like()`__:

```
query.filter(User.name.like("%ed%"))
```

> ##### Note
> 
> `ColumnOperators.like()` renders the `LIKE` operator, which is __case insensitive__ on _some backends_, and __case sensitive__ on _others_. For _guaranteed_ __case-insensitive comparisons__, use `ColumnOperators.ilike()`.

* __`ColumnOperators.ilike()` (case-insensitive LIKE)__:

```
query.filter(User.name.ilike("%ed%"))
```

> ##### Note
> 
> _most backends_ __don't support `ILIKE` directly__. For those, the `ColumnOperators.ilike()` operator _renders an expression_ __combining `LIKE` with the `LOWER`__ _SQL function_ applied to each operand.

* __`ColumnOperators.in_()`__:

```
from sqlalchemy import tuple_

query.filter(User.name.in_(["ed", "wendy", "jack"]))

query.filter(User.name.in_(session.query(User.name).filter(User.name.like("%ed%"))))

query.filter(
    tuple_(User.name, User.fullname).in_([("ed", "edsnickname"), ("wendy", "windy")])
)
```

* __`ColumnOperators.not_in()`__:

```
query.filter(~User.name.in_(["ed", "wendy", "jack"]))
```

* __`ColumnOperators.is_()`__:

```
query.filter(User.name == None)

query.filter(User.name.is_(None))
```

* __`ColumnOperators.is_not()`__:

```
query.filter(User.name != None)

query.filter(User.name.is_not(None))
```

* __`AND`__:
```
from sqlalchemy import and_

query.filter(and_(User.name == "ed", User.fullname == "Ed Jones"))

query.filter(User.name == "ed", User.fullname == "Ed Jones")

query.filter(User.name == "ed").filter(User.fullname == "Ed Jones")
```

> ##### Note
> 
> Make sure you _use_ `and_()` and __not the Python `and` operator__!

* __`OR`__:

```
from sqlalchemy import or_

query.filter(or_(User.name == "ed", User.name == "wendy"))
```

> ##### Note
> 
> Make sure you _use_ `or_()` and __not the Python `or` operator__!

* __`ColumnOperators.match()`__:

```
query.filter(User.name.match("wendy"))
```

> ##### Note
> 
> `ColumnOperators.match()` uses a _database-specific_ `MATCH` or `CONTAINS` _function_; its __behavior will vary by backend__ and is __not available__ on _some backends_ such as `SQLite`.

##### Returning Lists and Scalars

A _number of methods_ on `Query` __immediately issue SQL__ and __return a value containing loaded database results__. Here's a brief tour:

* __Query.all()__ returns a _list_.

In [36]:
query = session.query(User).filter(User.name.like("%ed%")).order_by(User.id)
print(f"{query.all() = }")

2022-10-16 11:49:15,279 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name LIKE ? ORDER BY users.id
2022-10-16 11:49:15,280 INFO sqlalchemy.engine.Engine [generated in 0.00195s] ('%ed%',)
query.all() = [<User(name=ed, fullname=Ed Jones, nickname=eddie)>, <User(name=fred, fullname=Fred Flintstone, nickname=freddy)>]


> ##### Warning
> 
> When the `Query` object _returns_ __lists of ORM-mapped objects__ such as the `User` object above, the entries are __deduplicated__ _based on_ `primary key`, as the results are __interpreted__ from the _SQL result set_. That is, if `SQL query` returns a _row_ with `id=7` _twice_, you would __only get a single `User(id=7)` object__ back in the _result list_. This __does not apply__ to the case when _individual columns_ are __queried__.

* __Query.first()__ __applies a limit of one__ and _returns_ the _first result as a scalar_.

In [37]:
print(f"{query.first() = }")

2022-10-16 12:03:19,515 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name LIKE ? ORDER BY users.id
 LIMIT ? OFFSET ?
2022-10-16 12:03:19,516 INFO sqlalchemy.engine.Engine [generated in 0.00159s] ('%ed%', 1, 0)
query.first() = <User(name=ed, fullname=Ed Jones, nickname=eddie)>


* __Query.one()__ _fully fetches all rows_, and if __not exactly__ `one object identity` or `composite row is present` in the result, __`raises an error`__. With multiple rows found.

In [38]:
try:
    user = query.one()
except Exception as e:
    print(str(e))

2022-10-16 12:05:23,338 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name LIKE ? ORDER BY users.id
2022-10-16 12:05:23,340 INFO sqlalchemy.engine.Engine [cached since 968.1s ago] ('%ed%',)
Multiple rows were found when exactly one was required


With no rows found.

In [39]:
try:
    user = query.filter(User.id == 99).one()
except Exception as e:
    print(str(e))

2022-10-16 12:06:28,007 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name LIKE ? AND users.id = ? ORDER BY users.id
2022-10-16 12:06:28,008 INFO sqlalchemy.engine.Engine [generated in 0.00148s] ('%ed%', 99)
No row was found when one was required


The `Query.one()` method is _great_ for systems that __expect to handle__ `"no items found"` versus `"multiple items found"` __differently__; such as a `RESTful web service`, which may want to raise a `"404 not found"` when __no results__ are found, but `raise an application error` when __multiple results__ are found.

* __Query.one_or_none()__ is _like_ `Query.one()`, except that _if no results are found_, it __doesn't raise an error__; it just __returns None__. Like `Query.one()`, however, it __does raise an error__ if _multiple results_ are found.

* __Query.scalar()__ _invokes_ the `Query.one()` method, and upon `success` _returns_ the __first column__ of the row.

In [40]:
query = session.query(User.id).filter(User.name == "ed").order_by(User.id)
print(f"{query.scalar() = }")

2022-10-16 12:13:12,040 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id 
FROM users 
WHERE users.name = ? ORDER BY users.id
2022-10-16 12:13:12,041 INFO sqlalchemy.engine.Engine [generated in 0.00172s] ('ed',)
query.scalar() = 1
